<a href="https://colab.research.google.com/github/O00O297/CCA-AI/blob/master/LoadWSI_to_Patch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install histomicstk --find-links https://girder.github.io/large_image_wheels
!pip install histomicstk
!pip install girder_client

In [ ]:
import girder_client
import numpy as np
from skimage.transform import resize
from matplotlib import pylab as plt
from matplotlib.colors import ListedColormap
from histomicstk.preprocessing.color_normalization import reinhard
from histomicstk.saliency.tissue_detection import (
    get_slide_thumbnail, get_tissue_mask)
from histomicstk.annotations_and_masks.annotation_and_mask_utils import (
    get_image_from_htk_response)
from histomicstk.preprocessing.color_normalization.\
    deconvolution_based_normalization import deconvolution_based_normalization
from histomicstk.preprocessing.color_deconvolution.\
    color_deconvolution import color_deconvolution_routine, stain_unmixing_routine
from histomicstk.preprocessing.augmentation.\
    color_augmentation import rgb_perturb_stain_concentration, perturb_stain_concentration

In [ ]:
import cv2
import os 

In [ ]:
APIURL = 'http://52.230.64.220:8080/api/v1/'
gc = girder_client.GirderClient(apiUrl=APIURL)

gc.authenticate(apiKey='LrdKJu7VQS4ItfhVIrQCXnWgt1pstHj611gwI3mf')

In [ ]:
def loadTile(SLIDE_ID,SLIDE_PATH,dir_Name):
  item_meta = gc.get('item/%s/tiles' % (SLIDE_ID))
  w = item_meta["sizeX"]
  h = item_meta["sizeY"]
  MAG = 5
  res = 250

  row = int(h*(MAG/40)/res)
  column = int(w*(MAG/40)/res)
  
  for i in range(0,row):
    for j in range(0,column):
      getStr = "/item/%s/tiles/region?left=%d&right=%d&top=%d&bottom=%d&units=mag_pixels" 
                % (SLIDE_ID, j*res, (j*res)+res, i*res, (i*res)+res) + "&magnification=%d" 
                % MAG +"&encoding=TIFF&tiffCompression=tiff_lzw"
      ImgTile = get_image_from_htk_response(gc.get(getStr, jsonResp=False))
      ImgTile = cv2.cvtColor(ImgTile, cv2.COLOR_RGB2BGR)
      Imgname = dir_Name +"-"+ str(i) + "_" + str(j) + ".tiff"
      status = threshold(ImgTile)
      if status:
        w = cv2.imwrite(SLIDE_PATH +"/"+ Imgname,ImgTile)
        # print (Imgname+ " is "+ str(w))
      

def threshold(Img):
  
  img = cv2.cvtColor(Img, cv2.COLOR_BGR2GRAY) 
  ret, thresh1 = cv2.threshold(img, 180, 255, cv2.THRESH_BINARY) 
  no_BPixel = sum(sum(1-(thresh1/255)))
  check = (250*250)/5
  if no_BPixel > check:
    return True
  else :
    return False

In [ ]:
#load all Slide in case
caseId = "5f111dbf6ba0d1144ddca1a7"
item_info = gc.get('item?folderId=%s&limit=%d&sort=lowerName' % (caseId,50))
rootpath = "/content/drive/Shared drives/CCA-AI Slide/"
Class = "CCA"
Case = "S63-04300"
rootpath = rootpath + Class + "/" + Case + "/"

parent_folder = []
for item in item_info:
  parent_folder.append([item["_id"],item["name"]]) #folder in case
print (parent_folder)

for p_folder in parent_folder:
  print(p_folder[1])
  directory = rootpath + p_folder[1] + "/"
  if not os.listdir(directory) :
    loadTile(p_folder[0],directory,p_folder[1])
  else :
    print("is not empty")
    continue

In [ ]:
#load by Slide
SLIDE_ID = "5f128ed8bf89248c33dd0390"
SLIDE_PATH = "/content/drive/Shared drives/CCA-AI Slide/HCC/S63-00819/S63-819-1H"
dir_Name = "S63-00819-1H"

loadTile(SLIDE_ID,SLIDE_PATH,dir_Name)